# 🛠️ التكليف الخامس: إصلاح الكود (Code Debugging)

## 🎯 الهدف
فحص الكود الذي يحتوي على خطأ، تحديد السبب، وكتابة الحل.

---

## 📋 الأخطاء الشائعة في مشاريع NLP

سنستعرض مجموعة من الأخطاء الشائعة التي قد تظهر في مشاريع معالجة اللغات الطبيعية وكيفية إصلاحها.

---

## 1️⃣ خطأ: نموذج Spacy غير موجود

In [1]:
# ❌ الكود الخاطئ
import spacy

# محاولة تحميل نموذج غير موجود
try:
    nlp = spacy.load("ar_core_news_sm")  # هذا النموذج قد لا يكون موجوداً
    print("تم تحميل النموذج")
except OSError as e:
    print(f"❌ خطأ: {e}")

❌ خطأ: [E050] Can't find model 'ar_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.


In [2]:
# ✅ الكود المصحح
import spacy

def load_spacy_model(model_name="xx_ent_wiki_sm"):
    """
    تحميل نموذج Spacy مع معالجة الأخطاء
    """
    try:
        nlp = spacy.load(model_name)
        print(f"✅ تم تحميل النموذج: {model_name}")
        return nlp
    except OSError:
        print(f"⚠️ النموذج {model_name} غير موجود، جاري التحميل...")
        import subprocess
        subprocess.run(["python", "-m", "spacy", "download", model_name])
        nlp = spacy.load(model_name)
        return nlp

# استخدام الدالة
nlp = load_spacy_model("xx_ent_wiki_sm")

✅ تم تحميل النموذج: xx_ent_wiki_sm


---

## 2️⃣ خطأ: مشكلة في الترميز (Encoding)

In [3]:
# ❌ الكود الخاطئ
import pandas as pd

try:
    # قراءة ملف عربي بدون تحديد الترميز
    df = pd.read_csv('arabic_file.csv')  # قد يسبب مشاكل
except Exception as e:
    print(f"❌ خطأ: {type(e).__name__}: {e}")

❌ خطأ: FileNotFoundError: [Errno 2] No such file or directory: 'arabic_file.csv'


In [4]:
# ✅ الكود المصحح
import pandas as pd

def read_arabic_csv(filepath):
    """
    قراءة ملف CSV عربي مع معالجة الترميز
    """
    encodings = ['utf-8', 'utf-8-sig', 'cp1256', 'iso-8859-6']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(filepath, encoding=encoding)
            print(f"✅ تم قراءة الملف بترميز: {encoding}")
            return df
        except (UnicodeDecodeError, UnicodeError):
            continue
    
    raise ValueError(f"فشل في قراءة الملف بجميع الترميزات المتاحة")

# الاستخدام:
# df = read_arabic_csv('arabic_file.csv')

---

## 3️⃣ خطأ: ValueError في TfidfVectorizer

In [5]:
# ❌ الكود الخاطئ
from sklearn.feature_extraction.text import TfidfVectorizer

texts = ["مرحبا", "", "أهلاً وسهلاً", None]  # يحتوي على فارغ و None

try:
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
except Exception as e:
    print(f"❌ خطأ: {type(e).__name__}: {e}")

❌ خطأ: AttributeError: 'NoneType' object has no attribute 'lower'


In [6]:
# ✅ الكود المصحح
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def clean_texts_for_vectorization(texts):
    """
    تنظيف النصوص قبل التحويل إلى متجهات
    """
    cleaned = []
    for text in texts:
        if text is None:
            cleaned.append('')
        elif isinstance(text, str):
            cleaned.append(text.strip())
        else:
            cleaned.append(str(text))
    return cleaned

# الاستخدام
texts = ["مرحبا", "", "أهلاً وسهلاً", None]
clean_texts = clean_texts_for_vectorization(texts)

# إزالة النصوص الفارغة أو استبدالها
clean_texts = [t if t else 'فارغ' for t in clean_texts]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_texts)
print(f"✅ تم التحويل بنجاح: {X.shape}")

✅ تم التحويل بنجاح: (4, 4)


---

## 4️⃣ خطأ: IndexError في التنبؤ

In [7]:
# ❌ الكود الخاطئ
from sklearn.linear_model import LogisticRegression
import numpy as np

# بيانات وهمية
X_train = np.random.rand(100, 10)
y_train = np.random.randint(0, 2, 100)

model = LogisticRegression()
model.fit(X_train, y_train)

# خطأ: تمرير نص بدلاً من متجه
try:
    new_text = "نص جديد للتنبؤ"
    prediction = model.predict(new_text)  # خطأ!
except Exception as e:
    print(f"❌ خطأ: {type(e).__name__}: {e}")

❌ خطأ: ValueError: Expected 2D array, got scalar array instead:
array=نص جديد للتنبؤ.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.


In [8]:
# ✅ الكود المصحح
def predict_text(text, model, vectorizer):
    """
    تنبؤ صحيح للنص الجديد
    """
    # التأكد من أن النص في قائمة
    if isinstance(text, str):
        text = [text]
    
    # تحويل إلى متجه
    X_new = vectorizer.transform(text)
    
    # التنبؤ
    prediction = model.predict(X_new)
    
    return prediction

# الاستخدام الصحيح:
# result = predict_text("نص جديد", model, vectorizer)

---

## 5️⃣ خطأ: MemoryError مع بيانات كبيرة

In [9]:
# ❌ الكود الخاطئ (قد يسبب MemoryError)
# تحميل جميع البيانات في الذاكرة مرة واحدة

# vectorizer = TfidfVectorizer(max_features=100000)  # عدد كبير جداً
# X = vectorizer.fit_transform(huge_dataset)  # بيانات ضخمة

In [10]:
# ✅ الكود المصحح
from sklearn.feature_extraction.text import TfidfVectorizer

def create_efficient_vectorizer(max_features=5000, min_df=5):
    """
    إنشاء vectorizer فعال للبيانات الكبيرة
    """
    vectorizer = TfidfVectorizer(
        max_features=max_features,  # تحديد الحد الأقصى
        min_df=min_df,              # تجاهل الكلمات النادرة
        max_df=0.95,                # تجاهل الكلمات الشائعة جداً
        dtype='float32'             # استخدام float32 بدلاً من float64
    )
    return vectorizer

# للبيانات الكبيرة جداً، استخدم batch processing
def batch_transform(texts, vectorizer, batch_size=1000):
    """
    معالجة البيانات على دفعات
    """
    from scipy.sparse import vstack
    
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_vec = vectorizer.transform(batch)
        results.append(batch_vec)
    
    return vstack(results)

print("✅ تم إنشاء دوال المعالجة الفعالة")

✅ تم إنشاء دوال المعالجة الفعالة


---

## 6️⃣ خطأ: Shape Mismatch في التدريب

In [11]:
# ❌ الكود الخاطئ
import numpy as np
from sklearn.linear_model import LogisticRegression

X = np.random.rand(100, 10)  # 100 عينة
y = np.random.randint(0, 2, 50)  # 50 تصنيف فقط!

try:
    model = LogisticRegression()
    model.fit(X, y)  # خطأ: عدم تطابق الأبعاد
except Exception as e:
    print(f"❌ خطأ: {type(e).__name__}: {e}")

❌ خطأ: ValueError: Found input variables with inconsistent numbers of samples: [100, 50]


In [12]:
# ✅ الكود المصحح
def validate_data_shapes(X, y):
    """
    التحقق من تطابق أبعاد البيانات
    """
    if hasattr(X, 'shape'):
        n_samples_X = X.shape[0]
    else:
        n_samples_X = len(X)
    
    n_samples_y = len(y)
    
    if n_samples_X != n_samples_y:
        raise ValueError(
            f"عدم تطابق: X يحتوي على {n_samples_X} عينة، "
            f"لكن y يحتوي على {n_samples_y} عينة"
        )
    
    print(f"✅ البيانات صحيحة: {n_samples_X} عينة")
    return True

# الاستخدام
X = np.random.rand(100, 10)
y = np.random.randint(0, 2, 100)
validate_data_shapes(X, y)

✅ البيانات صحيحة: 100 عينة


True

---

## 📋 قالب تقرير إصلاح الكود

In [13]:
fix_report_template = """
# 📋 تقرير إصلاح الكود

## معلومات الخطأ

| الحقل | القيمة |
|-------|--------|
| **نوع الخطأ** | [اسم الخطأ] |
| **رسالة الخطأ** | [نص الرسالة] |
| **السطر** | [رقم السطر] |
| **الملف** | [اسم الملف] |

---

## الكود الأصلي (الخاطئ)

```python
# الكود هنا
```

---

## سبب الخطأ

[شرح سبب حدوث الخطأ]

---

## الكود المصحح

```python
# الكود المصحح هنا
```

---

## التحقق من الإصلاح

- [ ] تم تشغيل الكود بنجاح
- [ ] لا توجد أخطاء
- [ ] النتائج صحيحة
"""

# حفظ القالب
with open('fix_report_template.md', 'w', encoding='utf-8') as f:
    f.write(fix_report_template)

print("✅ تم حفظ قالب التقرير في: fix_report_template.md")

✅ تم حفظ قالب التقرير في: fix_report_template.md


---

## 📌 ملخص التكليف الخامس

### الأخطاء التي تم تغطيتها:

| # | الخطأ | السبب | الحل |
|---|-------|-------|------|
| 1 | OSError: Model not found | نموذج غير مثبت | تحميل النموذج تلقائياً |
| 2 | UnicodeDecodeError | ترميز خاطئ | تجربة ترميزات متعددة |
| 3 | ValueError: empty vocabulary | نصوص فارغة | تنظيف البيانات |
| 4 | IndexError | نوع بيانات خاطئ | تحويل صحيح للمتجهات |
| 5 | MemoryError | بيانات ضخمة | batch processing |
| 6 | Shape mismatch | عدم تطابق أبعاد | التحقق من البيانات |

### المخرجات:
- `fix_report_template.md` - قالب تقرير الإصلاح

---

**✅ انتهى التكليف الخامس**